In [18]:
import xml.etree.ElementTree as et
import glob
import os
import json
from tqdm import tqdm

In [2]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]
def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]

In [30]:
classes = []
input_dir = "../data/final_dataset/valid_labels/"
output_dir = "../data/final_dataset/valid_yolo_labels/"
image_dir = "../data/final_dataset/valid_images/"

In [31]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [32]:
voc_xml_files = sorted(glob.glob(os.path.join(input_dir, '*.xml')))


for file in tqdm(voc_xml_files):
    basename = os.path.basename(file)
    filename = os.path.splitext(basename)[0]


    # if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
    #     print(f"{filename} image doesn't exist")

    result = []
    tree = et.parse(file)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find('name').text
        if label not in classes:
            classes.append(label)
        idx = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{idx} {bbox_string}")
    
    if result:
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))


100%|██████████| 42/42 [00:00<00:00, 5490.27it/s]


In [1]:
import glob
import os

dataset_path = "../data/final_dataset/"
image_type = "valid"
images_path = dataset_path + f"images/{image_type}/"
labels_path = dataset_path + f"labels/{image_type}/"
images = sorted(glob.glob(os.path.join(images_path, '*')))
labels = [f'{os.path.dirname(labels_path)}/{os.path.splitext(os.path.basename(img))[0]}.txt' for img in images]
outputs = []
for out in zip(images, labels):
    outputs.append(out[0])
    outputs.append(out[1])

In [2]:
with open(os.path.join(dataset_path, f"{image_type}_list.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(outputs))